In [144]:
from data import *
from utils.augmentations import SSDAugmentation
import torch
from torch.autograd import Variable
from box_utils import match, point_form, jaccard, match, intersect, encode
from SSD_generate_anchors import generate_ssd_priors

from pathlib import Path
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)

In [2]:
root = Path.home()/'Documents'/'DATASETS'/'VOCdevkit'
cfg = voc

In [3]:
dataset = VOCDetection(root = root,
                      transform=SSDAugmentation(cfg['min_dim'], MEANS))

In [4]:
len(dataset)

16551

In [5]:
sample = dataset[12706]
print(sample[0].size())
print(sample[1].shape)

torch.Size([3, 300, 300])
(4, 5)


In [6]:
import torch.utils.data as data

In [54]:
data_loader = data.DataLoader(dataset, 4,
                                  num_workers=2,
                                  shuffle=True,
                                  collate_fn=detection_collate,
                                  pin_memory=True)
batch_iterator = iter(data_loader)

# for i, d in enumerate(batch_iterator):
#     print(d[0])
#     print(d[1])
#     if i ==1:
#         break

In [8]:
import collections

SSDBoxSizes = collections.namedtuple('SSDBoxSizes', ['min', 'max'])

Spec = collections.namedtuple('Spec', ['feature_map_size', 'shrinkage', 'box_sizes', 
                                       'aspect_ratios'])

specs = [
    Spec(38, 8, SSDBoxSizes(30, 60), [2]),
    Spec(19, 16, SSDBoxSizes(60, 111), [2, 3]),
    Spec(10, 32, SSDBoxSizes(111, 162), [2, 3]),
    Spec(5, 64, SSDBoxSizes(162, 213), [2, 3]),
    Spec(3, 100, SSDBoxSizes(213, 264), [2]),
    Spec(1, 300, SSDBoxSizes(264, 315), [2])
]

priors = generate_ssd_priors(specs)

In [9]:
priors = torch.from_numpy(priors)
priors = priors.float()

In [10]:
priors_point_form = point_form(priors)
print(priors_point_form.size())

torch.Size([8732, 4])


In [11]:
default = priors_point_form.data

In [12]:
print(priors.size())


torch.Size([8732, 4])


In [13]:
a = torch.Tensor([[0,1,2],[3,4,5],[6,7,8]])
a.unsqueeze_(-1)
print(a)
print(a.size())

tensor([[[0.],
         [1.],
         [2.]],

        [[3.],
         [4.],
         [5.]],

        [[6.],
         [7.],
         [8.]]])
torch.Size([3, 3, 1])


In [14]:
a = a.expand(3,3,10)
print(a)
print(a.size())

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]],

        [[3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
         [4., 4., 4., 4., 4., 4., 4., 4., 4., 4.],
         [5., 5., 5., 5., 5., 5., 5., 5., 5., 5.]],

        [[6., 6., 6., 6., 6., 6., 6., 6., 6., 6.],
         [7., 7., 7., 7., 7., 7., 7., 7., 7., 7.],
         [8., 8., 8., 8., 8., 8., 8., 8., 8., 8.]]])
torch.Size([3, 3, 10])


In [132]:
for idx, (image, target) in enumerate(batch_iterator):
#     print(image.size(0))
    bs = image.size(0)
    for i in range(bs):
        truths = target[i][:,:-1].data
        label  = target[i][:,-1].data
#         print(label.size())
        inter = intersect(truths, default)
        iou = jaccard(truths, default)
#         print(inter.size())
#         print(union.size())
#         print(overlap)
    if idx == 0:
        break

1 8732
1 8732
1 8732
1 8732
5 8732
5 8732
5 8732
5 8732


In [169]:
label

tensor([14., 14., 14.,  1.,  1.])

In [170]:
best_prior_overlap, best_prior_idx = iou.max(1, keepdim=True)
print(best_prior_overlap)
print(best_prior_idx)

tensor([[0.5705],
        [0.5421],
        [0.7293],
        [0.6707],
        [0.6830]])
tensor([[8337],
        [6969],
        [7077],
        [7305],
        [7557]])


In [171]:
best_truth_overlap, best_truth_idx = iou.max(0, keepdim=True)
print(best_truth_overlap)
print(best_truth_idx)
print(best_truth_overlap.size())
print(best_truth_idx.size())

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0919, 0.1016, 0.1364]])
tensor([[4, 4, 4,  ..., 0, 0, 0]])
torch.Size([1, 8732])
torch.Size([1, 8732])


In [172]:
best_truth_idx.squeeze_(0)
best_truth_overlap.squeeze_(0)
print(best_truth_overlap.size())
print(best_truth_idx.size())

torch.Size([8732])
torch.Size([8732])


In [173]:
best_prior_idx.squeeze_(1)
best_prior_overlap.squeeze_(1)
print(best_prior_idx.size())
print(best_prior_overlap.size())

torch.Size([5])
torch.Size([5])


In [174]:
best_truth_overlap.index_fill_(0, best_prior_idx, 2)

tensor([0.0000, 0.0000, 0.0000,  ..., 0.0919, 0.1016, 0.1364])

In [175]:
print(best_truth_idx)
print(best_prior_idx)

tensor([4, 4, 4,  ..., 0, 0, 0])
tensor([8337, 6969, 7077, 7305, 7557])


In [176]:
label

tensor([14., 14., 14.,  1.,  1.])

In [177]:
a = truths[best_truth_idx]
print(a.size())

torch.Size([8732, 4])


In [24]:
# for i in a:
#     print(i)

In [115]:
# for j in range(best_prior_idx.size(0)):
#     print(best_truth_idx[best_prior_idx[j]])
#     best_truth_idx[best_prior_idx[j]] = j
#     print(best_truth_idx[best_prior_idx[j]])

In [178]:
loc_t = torch.Tensor(4, 8732, 4)
conf_t = torch.LongTensor(4, 8732)

In [185]:
for j in range(best_prior_idx.size(0)):
        best_truth_idx[best_prior_idx[j]] = j
matches = truths[best_truth_idx]          # Shape: [num_priors,4]
conf = label[best_truth_idx]          # Shape: [num_priors]
conf[best_truth_overlap < 0.6] = 0  # label as background
loc = encode(matches, point_form(priors), [0.1, 0.2])
loc_t[0] = loc    # [num_priors,4] encoded offsets to learn
conf_t[0] = conf

In [186]:
for a in conf:
    if a != 0:
        print(a)

tensor(14.)
tensor(14.)
tensor(14.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(14.)


In [205]:
pos = conf_t > 0
num_pos = pos.sum(dim=1, keepdim=True)


In [206]:
num_pos

tensor([[  11],
        [6312],
        [4061],
        [8726]])

In [207]:
pos.dim()


2

In [208]:
loc_data = torch.Tensor(4,8732,4)

In [209]:
pos_idx = pos.unsqueeze(pos.dim()).expand_as(loc_data)
print(pos_idx.size())

torch.Size([4, 8732, 4])


In [211]:
loc_p = loc_data[pos_idx].view(-1, 4)
print(loc_p.size())


torch.Size([19110, 4])


In [212]:
loc_t = loc_t[pos_idx].view(-1, 4)
print(loc_t.size())

torch.Size([19110, 4])


In [28]:
aa = torch.Tensor([1,2,4,5,6,7])
aa

tensor([1., 2., 4., 5., 6., 7.])

In [29]:
aa.index_fill_(0, torch.tensor([3]),9)

tensor([1., 2., 4., 9., 6., 7.])

In [30]:
iou.max(0, keepdim=True)

torch.return_types.max(
values=tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0209, 0.0310, 0.0175]]),
indices=tensor([[0, 0, 0,  ..., 0, 0, 0]]))

In [31]:
iou.size()

torch.Size([1, 8732])

In [108]:
iou_np = iou.numpy()
# print(iou_np)

In [60]:
d = np.amax(iou_np, axis= -1, keepdims=True)
print(d)

[[0.7423362 ]
 [0.717093  ]
 [0.6759641 ]
 [0.7175843 ]
 [0.76521665]
 [0.859685  ]
 [0.7188304 ]
 [0.738938  ]
 [0.69099313]]


In [62]:
np.argmax(iou_np, axis= -1).shape

(9,)

In [49]:
np.where(np.argmax(iou_np, axis = 1))

(array([0, 1]),)

In [52]:
ind = np.unravel_index(np.argmax(iou_np, axis = -1), iou_np.shape)
print(ind[1])

[6109  884]


In [107]:
# dd, ll = iou.max(0, keepdim=True)
# dd.squeeze_(0)
# dd = dd.numpy()
# # print(dd)

# ll.squeeze_(0)
# ll = ll.numpy()
# print(ll)

In [106]:
# dd_ = np.amax(iou_np, axis=0, )
# print(dd_)

In [76]:
sum(dd == dd_)

8732

In [105]:
ll_ = np.argmax(iou_np, axis=0)

In [10]:
from layers.modules import MultiBoxLoss
from ssd import build_ssd

In [11]:
cfg = voc

In [12]:
cfg

{'num_classes': 21,
 'lr_steps': (80000, 100000, 120000),
 'max_iter': 120000,
 'feature_maps': [38, 19, 10, 5, 3, 1],
 'min_dim': 300,
 'steps': [8, 16, 32, 64, 100, 300],
 'min_sizes': [30, 60, 111, 162, 213, 264],
 'max_sizes': [60, 111, 162, 213, 264, 315],
 'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
 'variance': [0.1, 0.2],
 'clip': True,
 'name': 'VOC'}

In [42]:
ssd_net = build_ssd('train', cfg['min_dim'], cfg['num_classes'])
net = ssd_net.cuda()

In [44]:
for i, d in enumerate(batch_iterator):
    image = d[0].cuda()
    out = net(image)
    if i == 0:
        break

In [45]:
print(out[0].size())
print(out[1].size())
print(out[2].size())


torch.Size([4, 8732, 4])
torch.Size([4, 8732, 21])
torch.Size([8732, 4])


In [50]:
criterion = MultiBoxLoss(cfg['num_classes'], 0.5, True, 0, True, 3, 0.5,
                             False, True)

/home/vlsilab/anaconda2/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  """Entry point for launching an IPython kernel.


In [51]:
# load train data
images, targets = next(batch_iterator)

In [52]:
images = Variable(images.cuda())
targets = [Variable(ann.cuda(), volatile=True) for ann in targets]

/home/vlsilab/anaconda2/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


In [53]:
out = net(images)

In [55]:
# criterion(out, targets)